In [87]:
using FermiCG
using PyCall
using Plots
using LinearAlgebra
using Printf

## Import PYSCF data

In [88]:
pyscf = pyimport("pyscf");
fcidump = pyimport("pyscf.tools.fcidump");
ctx = fcidump.read("fe2s2");
h = ctx["H1"];
g = ctx["H2"];
ecore = ctx["ECORE"];
g = pyscf.ao2mo.restore("1", g, size(h,2));

In [179]:
# blocking from paper
blocks = [[0,1],[2,3,4,5,6],[7,9,11],[8,10,12],[13,14,15,16,17],[18,19]]
init_fspace = [(2,2),(5,0),(3,3),(3,3),(0,5),(2,2)]

for b in blocks
    b .+= 1
end
clusters = [Cluster(i,collect(blocks[i])) for i = 1:length(blocks)]
display(clusters)

6-element Array{Cluster,1}:
 Cluster(1, [1, 2])
 Cluster(2, [3, 4, 5, 6, 7])
 Cluster(3, [8, 10, 12])
 Cluster(4, [9, 11, 13])
 Cluster(5, [14, 15, 16, 17, 18])
 Cluster(6, [19, 20])

In [93]:
# blocking 1
blocks = [  [0,1,2,3,6,7,8],        # L1(sig)+ Fe1(t) + L2(sig)
            [11,12,13,16,17,18,19], # L2(sig)+ Fe2(t) + L3(sig)
            [4,5,9,10,14,15],       # Fe1(e) + Fe2(e) + L2(pi)
         ]

init_fspace = ((7,4), (7,4), (4,4))

for b in blocks
    b .+= 1
end
clusters = [Cluster(i,collect(blocks[i])) for i = 1:length(blocks)]
display(clusters)


3-element Array{Cluster,1}:
 Cluster(1, [1, 2, 3, 4, 7, 8, 9])
 Cluster(2, [12, 13, 14, 17, 18, 19, 20])
 Cluster(3, [5, 6, 10, 11, 15, 16])

In [101]:
# blocking 2
blocks = [  [0,1],                     # L1(sig)
            [2,3,4,5,6,13,14,15,16,17],# Fe1 Fe2
            [7,8,9,10,11,12],          # L2(sig, pi)
            [18,19],                   # L3(sig)
         ]

init_fspace = [(2,2), (5,5), (6,6), (2,2)]

for b in blocks
    b .+= 1
end
clusters = [Cluster(i,collect(blocks[i])) for i = 1:length(blocks)]
display(clusters)

4-element Array{Cluster,1}:
 Cluster(1, [1, 2])
 Cluster(2, [3, 4, 5, 6, 7, 14, 15, 16, 17, 18])
 Cluster(3, [8, 9, 10, 11, 12, 13])
 Cluster(4, [19, 20])

In [180]:
ints = InCoreInts(ecore,h,g);

# U = FermiCG.block_diagonalize_orbitals(h,clusters);
# ints = FermiCG.orbital_rotation(ints,U);


na = 0
nb = 0
for f in init_fspace
    na += f[1]
    nb += f[2]
end
display(na)
display(nb)
# get initial guess for 1RDM
tmp = eigen(ints.h1).vectors;
rdm1a_guess = tmp[:,1:na]*tmp[:,1:na]';
rdm1b_guess = tmp[:,1:nb]*tmp[:,1:nb]';
# eigen(ints.h1).values


15

15

In [181]:
e_cmf, U, Da, Db  = FermiCG.cmf_oo(ints, clusters, init_fspace, rdm1a_guess, rdm1b_guess,
                                        max_iter_oo=40, verbose=0, gconv=1e-7, method="bfgs");
ints2 = FermiCG.orbital_rotation(ints,U);


 ooCMF Iter:    1 Total= -115.954595235557 Active= -115.954595235557 G=     5.72e-01
 ooCMF Iter:    2 Total= -116.428571798805 Active= -116.428571798805 G=     2.93e-01
 ooCMF Iter:    3 Total= -116.464915432928 Active= -116.464915432928 G=     1.67e-01
 ooCMF Iter:    4 Total= -116.484800876617 Active= -116.484800876617 G=     8.47e-02
 ooCMF Iter:    5 Total= -116.488093717023 Active= -116.488093717023 G=     5.87e-02
 ooCMF Iter:    6 Total= -116.489067542132 Active= -116.489067542132 G=     2.25e-02
 ooCMF Iter:    7 Total= -116.489494510348 Active= -116.489494510348 G=     1.37e-02
 ooCMF Iter:    8 Total= -116.489625121270 Active= -116.489625121270 G=     1.32e-02
 ooCMF Iter:    9 Total= -116.489701401702 Active= -116.489701401702 G=     6.24e-03
 ooCMF Iter:   10 Total= -116.489725793065 Active= -116.489725793065 G=     3.10e-03
 ooCMF Iter:   11 Total= -116.489729018542 Active= -116.489729018542 G=     1.36e-03
 ooCMF Iter:   12 Total= -116.489729904313 Active= -116.489729904

 * Status: success

 * Candidate solution
    Final objective value:     -1.164897e+02

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 1.32e-07 ≰ 0.0e+00
    |x - x'|/|x'|          = 3.47e-07 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.14e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 9.76e-16 ≰ 0.0e+00
    |g(x)|                 = 7.66e-08 ≤ 1.0e-07

 * Work counters
    Seconds run:   2  (vs limit Inf)
    Iterations:    29
    f(x) calls:    72
    ∇f(x) calls:   72


 ooCMF Iter:   30 Total= -116.489730286466 Active= -116.489730286466 G=     2.39e-07
*ooCMF -116.48973029 


# Compute `cluster_bases` and `cluster_ops`


In [182]:
max_roots = 100

#
# Build Cluster basis
cluster_bases = FermiCG.compute_cluster_eigenbasis(ints2, clusters, verbose=0, max_roots=max_roots,
        init_fspace=init_fspace, rdm1a=Da, rdm1b=Db);

In [183]:


#
# Build ClusteredOperator
clustered_ham = FermiCG.extract_ClusteredTerms(ints2, clusters);


# Build Cluster Operators
cluster_ops = FermiCG.compute_cluster_ops(cluster_bases, ints2);

# Add cmf hamiltonians for doing MP-style PT2 
FermiCG.add_cmf_operators!(cluster_ops, cluster_bases, ints2, Da, Db, verbose=0);

 Remove duplicates
 Number of terms reduced from  5226 to  2226


In [220]:
nroots = 7
ci_vector = FermiCG.ClusteredState(clusters, FermiCG.FockConfig(init_fspace), R=nroots);
FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, FermiCG.FockConfig([(2,2),(4,1),(3,3),(3,3),(1,4),(2,2)]),R=nroots))
FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, FermiCG.FockConfig([(2,2),(3,2),(3,3),(3,3),(2,3),(2,2)]),R=nroots))
FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, FermiCG.FockConfig([(2,2),(2,3),(3,3),(3,3),(3,2),(2,2)]),R=nroots))
FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, FermiCG.FockConfig([(2,2),(1,4),(3,3),(3,3),(4,1),(2,2)]),R=nroots))
FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, FermiCG.FockConfig([(2,2),(0,5),(3,3),(3,3),(5,0),(2,2)]),R=nroots))


for (fock,configs) in deepcopy(ci_vector.data)
    for ci in clusters
        for cj in clusters
            #a
            new_fock = [fock...]
            new_fock[ci.idx] = (new_fock[ci.idx][1]+1, new_fock[ci.idx][2])
            new_fock[cj.idx] = (new_fock[cj.idx][1]-1, new_fock[cj.idx][2])
            new_fock = FermiCG.FockConfig(new_fock)
            all([f[1]>=0 && f[1]<=length(clusters[ii]) for (ii,f) in enumerate(new_fock)]) || continue
            FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, new_fock,R=nroots))
        end
    end
    
    for ci in clusters
        for cj in clusters
            #b
            new_fock = [fock...]
            new_fock[ci.idx] = (new_fock[ci.idx][1], new_fock[ci.idx][2]+1)
            new_fock[cj.idx] = (new_fock[cj.idx][1], new_fock[cj.idx][2]-1)
            new_fock = FermiCG.FockConfig(new_fock)
            
            all([f[2]>=0 && f[2]<=length(clusters[ii]) for (ii,f) in enumerate(new_fock)]) || continue
            FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, new_fock,R=nroots))
        end
    end
end
        
display(ci_vector)


 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =    96  
 ----------                root ------:     =     1  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 --------------------------------------------------


In [221]:
e0, v0 = FermiCG.tpsci_ci(ci_vector, cluster_ops, clustered_ham,
                            thresh_cipsi=5e-3, # Threshold for adding to P-space
                            thresh_foi=1e-5,    # Threshold for keeping terms when defining FOIS    
                            thresh_asci=-1,     # Threshold of P-space configs to search from
                            max_iter=10,
                            matvec=3);


 ci_vector     :(96, 7)
 thresh_cipsi  :0.005
 thresh_foi    :1.0e-5
 thresh_asci   :-1
 max_iter      :10
 conv_thresh   :0.0001

     Selected CI Iteration:    1 epsilon:   0.00500000
 Build Hamiltonian matrix with dimension:    96
  0.236004 seconds (560.83 k allocations: 22.798 MiB)
  Root       Energy           S2
     1 -116.50551027  30.00000000
     2 -116.49938379  19.54034374
     3 -116.49665243   8.60907940
     4 -116.49545498   5.09000771
     5 -116.49189171   5.40131729
     6 -116.48785659   1.38005164
     7 -116.40382680  29.99999999
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =    96  
 ----------                root ------:     =     1  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.004               1                   ( 2,2 )( 5,0 )( 3

 0.001               1                   ( 2,2 )( 1,5 )( 3,3 )( 2,3 )( 5,0 )( 2,2 )
 0.001               1                   ( 2,2 )( 0,5 )( 3,2 )( 3,3 )( 5,1 )( 2,2 )
 --------------------------------------------------
 Length of ASCI vector       96 →       96 
 Norm of delta v:
   1.00000000
   1.00000000
   1.00000000
   1.00000000
   1.00000000
   1.00000000
   1.00000000
 In open_matvec_thread2

 Number of jobs:    2438
 Number of threads: 12
  2.307111 seconds (4.86 M allocations: 265.977 MiB, 12.74% gc time)
 Now collect thread results
  2.601129 seconds (6.39 M allocations: 343.935 MiB, 11.30% gc time)
 Length of FOIS vector: 372404
 Length of FOIS vector: 372308
 Compute diagonal
  0.407459 seconds (9.31 M allocations: 146.520 MiB)

  Root         E(0)         E(2)
     1 -116.50551027 -116.52748031
     2 -116.49938379 -116.52774244
     3 -116.49665243 -116.53086532
     4 -116.49545498 -116.53168361
     5 -116.49189171 -116.52878773
     6 -116.48785659 -116.52756058
    

 0.043               6                   ( 2,2 )( 4,2 )( 3,3 )( 2,3 )( 2,3 )( 2,2 )
 0.064               6                   ( 2,2 )( 3,2 )( 2,3 )( 3,3 )( 3,3 )( 2,2 )
 0.067               7                   ( 2,2 )( 3,2 )( 3,3 )( 2,3 )( 3,3 )( 2,2 )
 0.051               7                   ( 2,2 )( 3,3 )( 3,2 )( 3,3 )( 2,3 )( 2,2 )
 0.077               6                   ( 2,2 )( 3,3 )( 3,3 )( 3,2 )( 2,3 )( 2,2 )
 0.035               5                   ( 2,2 )( 3,2 )( 3,2 )( 3,3 )( 2,4 )( 2,2 )
 0.036               6                   ( 2,2 )( 3,2 )( 3,3 )( 3,2 )( 2,4 )( 2,2 )
 0.051               7                   ( 2,2 )( 3,3 )( 2,3 )( 3,3 )( 3,2 )( 2,2 )
 0.077               6                   ( 2,2 )( 3,3 )( 3,3 )( 2,3 )( 3,2 )( 2,2 )
 0.035               5                   ( 2,2 )( 2,3 )( 2,3 )( 3,3 )( 4,2 )( 2,2 )
 0.036               6                   ( 2,2 )( 2,3 )( 3,3 )( 2,3 )( 4,2 )( 2,2 )
 0.028               6                   ( 2,2 )( 2,4 )( 3,2 )( 3,3 )( 3,2 )

 0.052               135                 ( 2,2 )( 1,4 )( 3,3 )( 3,3 )( 4,1 )( 2,2 )
 --------------------------------------------------
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =  2325  
 ----------                root ------:     =     7  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.009               23                  ( 2,2 )( 4,1 )( 2,3 )( 3,3 )( 2,4 )( 2,2 )
 0.009               27                  ( 2,2 )( 4,1 )( 3,3 )( 2,3 )( 2,4 )( 2,2 )
 0.006               27                  ( 2,2 )( 4,2 )( 3,2 )( 3,3 )( 1,4 )( 2,2 )
 0.011               26                  ( 2,2 )( 4,2 )( 3,3 )( 3,2 )( 1,4 )( 2,2 )
 0.027               27                  ( 2,2 )( 4,2 )( 2,3 )( 3,3 )( 2,3 )( 2,2 )
 0.040               33                  ( 2,2 )( 4,2 )( 3,3 

 ---------- Fockspaces in state ------: Dim =  3277  
 ----------                root ------:     =     6  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.052               271                 ( 2,2 )( 4,1 )( 3,3 )( 3,3 )( 1,4 )( 2,2 )
 0.433               110                 ( 2,2 )( 3,2 )( 3,3 )( 3,3 )( 2,3 )( 2,2 )
 0.432               122                 ( 2,2 )( 2,3 )( 3,3 )( 3,3 )( 3,2 )( 2,2 )
 0.051               270                 ( 2,2 )( 1,4 )( 3,3 )( 3,3 )( 4,1 )( 2,2 )
 --------------------------------------------------
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =  3277  
 ----------                root ------:     =     7  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              #

...

 ---------- Fockspaces in state ------: Dim =  5178  
 ----------                root ------:     =     5  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.021               414                 ( 2,2 )( 4,1 )( 3,3 )( 3,3 )( 1,4 )( 2,2 )
 0.436               516                 ( 2,2 )( 3,2 )( 3,3 )( 3,3 )( 2,3 )( 2,2 )
 0.422               534                 ( 2,2 )( 2,3 )( 3,3 )( 3,3 )( 3,2 )( 2,2 )
 0.020               432                 ( 2,2 )( 1,4 )( 3,3 )( 3,3 )( 4,1 )( 2,2 )
 0.027               15                  ( 2,2 )( 2,2 )( 3,3 )( 3,3 )( 3,3 )( 2,2 )
 0.002               60                  ( 2,2 )( 3,2 )( 2,3 )( 3,3 )( 3,3 )( 2,2 )
 0.002               55                  ( 2,2 )( 3,2 )( 3,3 )( 2,3 )( 3,3 )( 2,2 )
 0.002               51                  ( 2,2 )( 3,3 )( 3,2 )( 3,3 )( 2,3 )( 2,2

  1.816245 seconds (78.46 M allocations: 1.827 GiB)
  Root       Energy           S2
     1 -116.55994950   3.96578748
     2 -116.55984176   3.95748508
     3 -116.54290408  10.96524492
     4 -116.54265437  11.70694477
     5 -116.52996976  11.36641095
     6 -116.52369702  27.90580334
     7 -116.46657786  19.90901239
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =  5283  
 ----------                root ------:     =     1  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.811               1                   ( 2,2 )( 5,0 )( 3,3 )( 3,3 )( 0,5 )( 2,2 )
 0.020               420                 ( 2,2 )( 4,1 )( 3,3 )( 3,3 )( 1,4 )( 2,2 )
 0.039               14                  ( 2,2 )( 4,0 )( 3,3 )( 3,3 )( 1,5 )( 2,2 )
 0.007               10                  (

 0.003               10                  ( 2,2 )( 4,1 )( 3,3 )( 2,3 )( 3,4 )( 1,2 )
 0.002               14                  ( 2,2 )( 4,3 )( 3,2 )( 3,2 )( 1,4 )( 2,2 )
 --------------------------------------------------
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =  5283  
 ----------                root ------:     =     5  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.021               420                 ( 2,2 )( 4,1 )( 3,3 )( 3,3 )( 1,4 )( 2,2 )
 0.436               559                 ( 2,2 )( 3,2 )( 3,3 )( 3,3 )( 2,3 )( 2,2 )
 0.421               566                 ( 2,2 )( 2,3 )( 3,3 )( 3,3 )( 3,2 )( 2,2 )
 0.020               437                 ( 2,2 )( 1,4 )( 3,3 )( 3,3 )( 4,1 )( 2,2 )
 0.028               15                  ( 2,2 )( 2,2 )( 3,3 

  2.502393 seconds (79.89 M allocations: 1.857 GiB, 24.57% gc time)
  Root       Energy           S2
     1 -116.55995184   3.96175232
     2 -116.55994770   3.95273009
     3 -116.54299495  10.97897051
     4 -116.54273832  11.70911401
     5 -116.53006459  11.36702002
     6 -116.52370069  27.89819868
     7 -116.46669608  19.90595989
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =  5324  
 ----------                root ------:     =     1  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.810               1                   ( 2,2 )( 5,0 )( 3,3 )( 3,3 )( 0,5 )( 2,2 )
 0.021               421                 ( 2,2 )( 4,1 )( 3,3 )( 3,3 )( 1,4 )( 2,2 )
 0.039               14                  ( 2,2 )( 4,0 )( 3,3 )( 3,3 )( 1,5 )( 2,2 )
 0.007               10   

 0.003               15                  ( 2,2 )( 4,3 )( 3,2 )( 3,2 )( 1,4 )( 2,2 )
 --------------------------------------------------
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =  5324  
 ----------                root ------:     =     5  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.021               421                 ( 2,2 )( 4,1 )( 3,3 )( 3,3 )( 1,4 )( 2,2 )
 0.435               577                 ( 2,2 )( 3,2 )( 3,3 )( 3,3 )( 2,3 )( 2,2 )
 0.421               572                 ( 2,2 )( 2,3 )( 3,3 )( 3,3 )( 3,2 )( 2,2 )
 0.020               444                 ( 2,2 )( 1,4 )( 3,3 )( 3,3 )( 4,1 )( 2,2 )
 0.028               15                  ( 2,2 )( 2,2 )( 3,3 )( 3,3 )( 3,3 )( 2,2 )
 0.002               61                  ( 2,2 )( 3,2 )( 2,3 

In [223]:
vec0 = FermiCG.get_vectors(v0)[:,1:2]

5324×2 Array{Float64,2}:
 -0.900166      0.0228317
 -0.0984252     0.00244469
  0.00496002   -0.000127226
 -0.00634928    0.000162883
  0.0139252    -0.000352556
 -0.00526423    0.000135043
  0.0214817    -0.000543947
 -0.0199608     0.000505359
  0.00111256   -2.8765e-5
  0.0139249    -0.000352547
  0.0213145    -0.000539689
  0.00638702   -0.00016379
 -0.0192479     0.000487312
  ⋮            
  8.28826e-5   -2.46985e-6
  0.00158034   -3.94338e-5
  0.00125532   -3.11935e-5
  0.000983962  -2.44007e-5
 -2.86613e-5   -5.84461e-5
 -5.94589e-5   -2.94586e-5
 -3.15957e-5   -5.89369e-5
  6.02931e-5    2.50751e-5
  5.80163e-5    2.87278e-5
  2.73901e-5    5.97211e-5
  2.82165e-5    5.98735e-5
 -6.00469e-5   -2.55055e-5

In [237]:
ci_vector = FermiCG.ClusteredState(clusters, FermiCG.FockConfig(init_fspace), R=2);
for (fock,configs) in v0.data
    FermiCG.add_fockconfig!(ci_vector,fock)
end
for (fock,configs) in v0.data
    for (config,coeffs) in configs
         ci_vector[fock][config] = coeffs[1:2]
    end
end
FermiCG.clip!(ci_vector,thresh=1e-3)
display(ci_vector)

 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =  1903  
 ----------                root ------:     =     1  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.810               1                   ( 2,2 )( 5,0 )( 3,3 )( 3,3 )( 0,5 )( 2,2 )
 0.021               307                 ( 2,2 )( 4,1 )( 3,3 )( 3,3 )( 1,4 )( 2,2 )
 0.039               14                  ( 2,2 )( 4,0 )( 3,3 )( 3,3 )( 1,5 )( 2,2 )
 0.007               9                   ( 2,2 )( 5,0 )( 2,3 )( 3,3 )( 1,5 )( 2,2 )
 0.008               10                  ( 2,2 )( 5,0 )( 3,3 )( 2,3 )( 1,5 )( 2,2 )
 0.007               10                  ( 2,2 )( 5,1 )( 3,2 )( 3,3 )( 0,5 )( 2,2 )
 0.007               10                  ( 2,2 )( 5,1 )( 3,3 )( 3,2 )( 0,5 )( 2,2 )
 0.038               15      

In [238]:
e1, v1 = FermiCG.tpsci_ci(ci_vector, cluster_ops, clustered_ham,
                            thresh_cipsi=5e-3, # Threshold for adding to P-space
                            thresh_foi=1e-5,    # Threshold for keeping terms when defining FOIS    
                            thresh_asci=-1,     # Threshold of P-space configs to search from
                            max_iter=10,
                            matvec=3);

 ci_vector     :(1903, 2)
 thresh_cipsi  :0.005
 thresh_foi    :1.0e-5
 thresh_asci   :-1
 max_iter      :10
 conv_thresh   :0.0001

     Selected CI Iteration:    1 epsilon:   0.00500000
 Build Hamiltonian matrix with dimension:  1903
  0.280677 seconds (12.39 M allocations: 291.697 MiB)
  Root       Energy           S2
     1 -116.55990039   4.14701352
     2 -116.55989725   4.13368420
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =  1903  
 ----------                root ------:     =     1  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.815               1                   ( 2,2 )( 5,0 )( 3,3 )( 3,3 )( 0,5 )( 2,2 )
 0.017               307                 ( 2,2 )( 4,1 )( 3,3 )( 3,3 )( 1,4 )( 2,2 )
 0.039               14                  ( 2,2 )( 4,0 )( 3

 Length of ASCI vector     2031 →     2031 
 Norm of delta v:
   1.99999330
   0.00440156
 In open_matvec_thread2

 Number of jobs:    4722
 Number of threads: 12
  6.467661 seconds (15.62 M allocations: 799.415 MiB)
 Now collect thread results
  6.663139 seconds (17.90 M allocations: 905.408 MiB)
 Length of FOIS vector: 1116464
 Length of FOIS vector: 1114433
 Compute diagonal
  1.520734 seconds (27.86 M allocations: 442.449 MiB, 23.39% gc time)

  Root         E(0)         E(2)
     1 -116.55990589 -116.57740244
     2 -116.55990003 -116.57790673
 Length of PT1  vector  1114433 →       38 
*TPSCI Iter 2   Dim: 2031   E(var): -116.55990589 -116.55990003 


In [ ]:
e2, v2 = FermiCG.tpsci_ci(v1, cluster_ops, clustered_ham,
                            thresh_cipsi=1e-3, # Threshold for adding to P-space
                            thresh_foi=1e-5,    # Threshold for keeping terms when defining FOIS    
                            thresh_asci=-1,     # Threshold of P-space configs to search from
                            max_iter=10,
                            matvec=3);

 ci_vector     :(2031, 2)
 thresh_cipsi  :0.001
 thresh_foi    :1.0e-5
 thresh_asci   :-1
 max_iter      :10
 conv_thresh   :0.0001

     Selected CI Iteration:    1 epsilon:   0.00100000
 Build Hamiltonian matrix with dimension:  2031
  0.340970 seconds (14.07 M allocations: 328.119 MiB)
  Root       Energy           S2
     1 -116.55990589   4.14527688
     2 -116.55990003   4.13209566
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =  2031  
 ----------                root ------:     =     1  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.815               1                   ( 2,2 )( 5,0 )( 3,3 )( 3,3 )( 0,5 )( 2,2 )
 0.017               338                 ( 2,2 )( 4,1 )( 3,3 )( 3,3 )( 1,4 )( 2,2 )
 0.039               14                  ( 2,2 )( 4,0 )( 3

 Length of ASCI vector     7226 →     7226 
 Norm of delta v:
   0.18142531
   0.18109152
 In open_matvec_thread2

 Number of jobs:    5860
 Number of threads: 12
 24.019578 seconds (44.42 M allocations: 1.838 GiB)
 Now collect thread results
 24.579760 seconds (48.44 M allocations: 2.002 GiB, 0.89% gc time)
 Length of FOIS vector: 1612204
 Length of FOIS vector: 1604978
 Compute diagonal
  2.116167 seconds (40.12 M allocations: 636.850 MiB, 21.77% gc time)

  Root         E(0)         E(2)
     1 -116.58119533 -116.59159360
     2 -116.58116624 -116.59150665
 Length of PT1  vector  1604978 →     3817 
 TPSCI Iter 2   Dim: 7226   E(var): -116.58119533 -116.58116624 

     Selected CI Iteration:    3 epsilon:   0.00100000
 Clip values < -1.0e+00           7226 →   7226
 Add pt vector to current space   7226 →  11043
 Build Hamiltonian matrix with dimension: 11043
  9.811819 seconds (252.08 M allocations: 6.152 GiB, 4.33% gc time)
  Root       Energy           S2
     1 -116.58380808   4

 13.905703 seconds (340.51 M allocations: 8.192 GiB, 2.74% gc time)
  Root       Energy           S2
     1 -116.58417476   4.20622866
     2 -116.58413247   4.20128784
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim = 12772  
 ----------                root ------:     =     1  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.731               1                   ( 2,2 )( 5,0 )( 3,3 )( 3,3 )( 0,5 )( 2,2 )
 0.020               610                 ( 2,2 )( 4,1 )( 3,3 )( 3,3 )( 1,4 )( 2,2 )
 0.050               23                  ( 2,2 )( 4,0 )( 3,3 )( 3,3 )( 1,5 )( 2,2 )
 0.012               14                  ( 2,2 )( 5,0 )( 2,3 )( 3,3 )( 1,5 )( 2,2 )
 0.011               15                  ( 2,2 )( 5,0 )( 3,3 )( 2,3 )( 1,5 )( 2,2 )
 0.011               15     

 Length of ASCI vector    14901 →    14901 
 Norm of delta v:
   1.99984184
   0.00964820
 In open_matvec_thread2

 Number of jobs:    5967
 Number of threads: 12
